In [1]:
%autosave 300
%autoreload 2
%reload_ext autoreload
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run/code/Users/soutrik.chowdhury/advanced_vision_modules_pytorch


In [22]:
from src.helper import set_seed, plot_loss_accuracy, get_device
from src.preprocessing import data_augmentations
from src.dataloader import get_data_loader
from src.custom_nets.custom_resnet import ResnetNetwork
from src.trainer import (
    train_module,
    test_module,
    model_drivers,
    suggestd_lr,
    lr_scheduler,
)
import torchvision.datasets as datasets
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from torchinfo import summary
from torchview import draw_graph

In [15]:
# Setting up basic training elements
set_seed(42)
device = get_device()
print(device)
batch_size = 512
epochs = 25
learning_rate = 1e-3

Random seed set as 42
cuda


In [16]:
# Specify the image classes
classes = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]
num_classes = len(classes)

In [17]:
# get CIFAR10 data from torchvision
# downloading train and test data
train = datasets.CIFAR10("./data", train=True, download=True)
test = datasets.CIFAR10("./data", train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [20]:
# get dataloader for data batching
dataloader_args = (
    dict(shuffle=True, batch_size=batch_size, num_workers=4, pin_memory=True)
    if device.type == "cuda"
    else dict(shuffle=True, batch_size=batch_size)
)

# get train and test transformations
train_transforms, test_transforms = data_augmentations()

# batch loading
train_loader, test_loader = get_data_loader(
    train, test, train_transforms, test_transforms, dataloader_args
)

In [21]:
for images, labels in train_loader:
    print(images.shape)
    print(labels.shape)
    print(images.mean())
    print(images.std())
    break

/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([512, 3, 32, 32])
torch.Size([512])
tensor(-0.0673)
tensor(1.3094)


## Basic Model without any LR scheduling

In [26]:
resnet_model = ResnetNetwork(in_channels=3, num_classes=num_classes).to(device)
summary(resnet_model, input_size=(1,3, 32, 32))

Layer (type:depth-idx)                   Output Shape              Param #
ResnetNetwork                            [1, 10]                   --
├─Sequential: 1-1                        [1, 64, 32, 32]           --
│    └─Conv2d: 2-1                       [1, 64, 32, 32]           1,728
│    └─BatchNorm2d: 2-2                  [1, 64, 32, 32]           128
│    └─ReLU: 2-3                         [1, 64, 32, 32]           --
├─ResnetBlock: 1-2                       [1, 128, 16, 16]          --
│    └─Sequential: 2-4                   [1, 128, 16, 16]          --
│    │    └─Conv2d: 3-1                  [1, 128, 32, 32]          73,728
│    │    └─MaxPool2d: 3-2               [1, 128, 16, 16]          --
│    │    └─BatchNorm2d: 3-3             [1, 128, 16, 16]          256
│    │    └─ReLU: 3-4                    [1, 128, 16, 16]          --
│    └─Sequential: 2-5                   [1, 128, 16, 16]          --
│    │    └─Conv2d: 3-5                  [1, 128, 16, 16]          73,728
│ 

In [28]:
model, optimizer, criterion, metric, early_stopping = model_drivers(
    model=resnet_model,
    learning_rate=learning_rate,
    device=device,
    num_classes=num_classes,
    model_name="resnet_model_base",
)

In [29]:
train_losses = []
train_metrics = []
test_losses = []
test_metrics = []

for epoch in range(0, epochs):
    print(f"Epoch {epoch}")
    train_losses, train_metrics = train_module(
        resnet_model,
        device,
        train_loader,
        optimizer,
        criterion,
        metric,
        train_losses,
        train_metrics,
    )
    test_losses, test_metrics = test_module(
        resnet_model,
        device,
        test_loader,
        criterion,
        metric,
        test_losses,
        test_metrics,
    )


    early_stopping(
        test_losses[-1], resnet_model, epoch
    )  # last recorded test loss to measure the improvement against the prior one
    if early_stopping.early_stop:
        print("Early stopping")
        break

Epoch 0


  0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           | 0/98 [00:00<?, ?it/s]/